# Test Results

In this notebook the results on the test split are generated for the 4 different models

### imports

In [1]:
from ultralytics import YOLO
import pandas as pd

## Evaluate

In [2]:
def evaluate_model(model_path, dataset_path, includesBalls):
    # load model
    model = YOLO(model_path)

    # validate
    metrics = model.val(data=dataset_path, imgsz = 640, project="evaluation", device="cuda:1", split="test", batch=64)

    # get metrics for both classes (person 1 and ball 0)
    metrics_total = metrics.results_dict

    # calculate f1 score total and for both classes
    f1_total = 2 * (metrics_total["metrics/precision(B)"] * metrics_total["metrics/recall(B)"]) / (metrics_total["metrics/precision(B)"] + metrics_total["metrics/recall(B)"])
    map50_total = metrics_total["metrics/mAP50(B)"]
    
    if includesBalls:
        results_person = metrics.class_result(1)
        results_ball = metrics.class_result(0)
        f1_person = 2 * (results_person[0] * results_person[1]) / (results_person[0] + results_person[1])
        f1_ball = 2 * (results_ball[0] * results_ball[1]) / (results_ball[0] + results_ball[1])
        map50_person = results_person[2]
        map50_ball = results_ball[2]

        predictions = get_ball_predictions(model)

        return round(f1_total, 4), round(map50_total, 4), round(f1_person, 4), round(map50_person, 4), round(f1_ball, 4), round(map50_ball, 4), predictions
    else:
        results_person = metrics.class_result(0)
        f1_person = 2 * (results_person[0] * results_person[1]) / (results_person[0] + results_person[1])
        map50_person = results_person[2]
        return round(f1_total, 4), round(map50_total, 4), round(f1_person, 4), round(map50_person, 4), None, None, None

def get_ball_predictions(model):
    # Initialize an empty list to store results
    data = []
    
    # Run inference
    results = model.predict(
        source="./datasets/YOLOPlayerBall/images/test",
        conf=0.3,       
        save=False,
        stream=True,
        batch=16,
        device="cuda:1",
        verbose=False
    )
    
    # Process results
    for result in results:
        # Initialize an empty list for detections in the current image
        detections = []
    
        boxes = result.boxes  # Bounding box outputs
        if boxes is not None:
            for box in boxes:
                # Extract information for each detected object
                bbox = box.xyxy[0].tolist()  # Bounding box (x1, y1, x2, y2)
                confidence = box.conf[0].item()  # Confidence score
                class_label = int(box.cls[0].item())  # Class label
                if class_label == 0:
                    # Add to the results list
                    detections.append({
                        "x1": bbox[0],
                        "y1": bbox[1],
                        "x2": bbox[2],
                        "y2": bbox[3],
                        "confidence": confidence,
                        "class_label": class_label
                    })
    
        data.append({
            "path": result.path.split("/")[-1],
            "detections": detections  # Store all detections for the image
        })
    
    return data

In [ ]:
models = [
            {"name": "YOLOv11 Just Player",
             "path": "./player-tracking/train_yolo11/weights/best.pt",
             "dataset": "datasets/YOLOPlayer/annotations.yaml",
             "includesBalls": False},
            {"name": "YOLOv11 Player and Ball",
             "path": "./player-ball-tracking/train_PreTrained_Ball_V11/weights/best.pt",
             "dataset": "datasets/YOLOPlayerBall/annotations.yaml",
             "includesBalls": True},
            {"name": "YOLOv5 Just Player",
             "path": "./player-tracking/train_yolov5/weights/best.pt",
             "dataset": "datasets/YOLOPlayer/annotations.yaml",
             "includesBalls": False},
            {"name": "YOLOv5 Player and Ball",
             "path": "./player-ball-tracking/train_Pretrained_Ball_V5/weights/best.pt",
             "dataset": "datasets/YOLOPlayerBall/annotations.yaml",
             "includesBalls": True},
        ]

results = []
for model in models:
    f1_total, map50_total, f1_person, map50_person, f1_ball, map50_ball, predictions = evaluate_model(model["path"], model["dataset"], model["includesBalls"])
    results.append({"model": model["name"], "f1_total": f1_total, "map50_total": map50_total, "f1_person": f1_person, "map50_person": map50_person, "f1_ball": f1_ball, "map50_ball": map50_ball, "predictions": predictions})


Ultralytics 8.3.31 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:1 (NVIDIA GeForce RTX 4090, 24111MiB)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /home/bevo/PlayerTracking/datasets/YOLOPlayer/labels/test.cache... 1963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1963/1963 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:05<00:00,  5.60it/s]


                   all       1963       3834      0.943      0.946      0.982      0.713
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to evaluation/val
Ultralytics 8.3.31 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:1 (NVIDIA GeForce RTX 4090, 24111MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /home/bevo/PlayerTracking/datasets/YOLOPlayerBall/labels/test.cache... 1963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1963/1963 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:06<00:00,  4.95it/s]


                   all       1963       5471      0.856      0.784      0.809      0.618
                  ball       1637       1637      0.748      0.575      0.627      0.465
                person       1963       3834      0.965      0.993      0.991      0.771
Speed: 0.2ms preprocess, 0.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to evaluation/val2
Ultralytics 8.3.31 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:1 (NVIDIA GeForce RTX 4090, 24111MiB)
YOLOv5s summary (fused): 193 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs


val: Scanning /home/bevo/PlayerTracking/datasets/YOLOPlayer/labels/test.cache... 1963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1963/1963 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:05<00:00,  5.68it/s]


                   all       1963       3834      0.926      0.994      0.967      0.778
Speed: 0.2ms preprocess, 0.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to evaluation/val3
Ultralytics 8.3.31 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:1 (NVIDIA GeForce RTX 4090, 24111MiB)
YOLOv5s summary (fused): 193 layers, 9,112,310 parameters, 0 gradients, 23.8 GFLOPs


val: Scanning /home/bevo/PlayerTracking/datasets/YOLOPlayerBall/labels/test.cache... 1963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1963/1963 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:06<00:00,  5.05it/s]


                   all       1963       5471      0.927      0.769      0.825      0.604
                  ball       1637       1637      0.894      0.561      0.667      0.465
                person       1963       3834       0.96      0.976      0.984      0.743
Speed: 0.2ms preprocess, 0.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to evaluation/val4


In [4]:
df = pd.DataFrame(results)
df

,model,f1_total,map50_total,f1_person,map50_person,f1_ball,map50_ball,predictions
0,YOLOv11 Just Player,0.9445,0.9821,0.9445,0.9821,NaN,NaN,None
1,YOLOv11 Player and Ball,0.8187,0.8090,0.9788,0.9908,0.6504,0.6271,"[{'path': 'New_Video_1_clip_6_100.jpg', 'detec..."
2,YOLOv5 Just Player,0.9588,0.9675,0.9588,0.9675,NaN,NaN,None
3,YOLOv5 Player and Ball,0.8403,0.8254,0.9678,0.9836,0.6896,0.6672,"[{'path': 'New_Video_1_clip_6_100.jpg', 'detec..."


In [5]:
df.to_csv("test_results.csv")